Name: Mihir Gawale
Roll no. 21102B0038
BE CMPN-B
Batch 2

In [1]:
import nltk

In [2]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [3]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [4]:
corpus="Martin, Justin can watch will.Spot will watch Martin.Will Justin spot Martin. Martin will pat Spot."

In [5]:
modified_corpus=""
modified_corpus = corpus.replace(',', '').replace('?', '')
print('Without punctuation string:', modified_corpus)

Without punctuation string: Martin Justin can watch will.Spot will watch Martin.Will Justin spot Martin. Martin will pat Spot.


In [6]:
modified_corpus2 = ""
sentences = modified_corpus.split('.')
for sentence in sentences:
    sentence = sentence.strip()
    if sentence:
        modified_corpus2 += f"<s> {sentence} </s> "

modified_corpus2 = modified_corpus2.strip()

print(modified_corpus2)

<s> Martin Justin can watch will </s> <s> Spot will watch Martin </s> <s> Will Justin spot Martin </s> <s> Martin will pat Spot </s>


In [7]:
from collections import defaultdict

tagged_sentences = [
    [('Martin', 'N'), ('Justin', 'N'), ('can', 'M'), ('watch', 'V'), ('will', 'N')],
    [('Spot', 'N'), ('will', 'M'), ('watch', 'V'), ('Martin', 'N')],
    [('Will', 'M'), ('Justin', 'N'), ('spot', 'V'), ('Martin', 'N')],
    [('Martin', 'N'), ('will', 'M'), ('pat', 'V'), ('Spot', 'N')]
]

emission_matrix = defaultdict(lambda: defaultdict(int))
transition_matrix = defaultdict(lambda: defaultdict(int))

for sentence in tagged_sentences:
    previous_tag = '<s>'
    for word, tag in sentence:
        emission_matrix[tag][word] += 1

        transition_matrix[previous_tag][tag] += 1

        previous_tag = tag
    transition_matrix[previous_tag]['</s>'] += 1

def normalize_matrix(matrix):
    normalized_matrix = {}
    for from_state, to_states in matrix.items():
        total = sum(to_states.values())
        normalized_matrix[from_state] = {to_state: count / total for to_state, count in to_states.items()}
    return normalized_matrix

normalized_emission_matrix = normalize_matrix(emission_matrix)
normalized_transition_matrix = normalize_matrix(transition_matrix)

def print_matrix(matrix, row_labels, col_labels, title):
    print(f"\n{title}:")
    print(f"{'':15}", end="")
    for col in col_labels:
        print(f"{col:15}", end="")
    print()
    for row in row_labels:
        print(f"{row:15}", end="")
        for col in col_labels:
            value = matrix.get(row, {}).get(col, 0)
            print(f"{value:<15.4f}", end="")
        print()

all_tags = list(set(tag for sentence in tagged_sentences for _, tag in sentence)) + ['<s>', '</s>']

print_matrix(normalized_emission_matrix, all_tags,
             list(set(word for sentence in tagged_sentences for word, _ in sentence)),
             "Emission Matrix")

print_matrix(normalized_transition_matrix, all_tags, all_tags, "Transition Matrix")
def tag_sentence(sentence):
    tags = []
    previous_probabilities = defaultdict(float)
    previous_probabilities['<s>'] = 1.0
    previous_tag = '<s>'

    for word in sentence:
        current_probabilities = defaultdict(float)
        best_tags = {}

        for tag in normalized_emission_matrix.keys():
            transition_prob = normalized_transition_matrix[previous_tag].get(tag, 0)
            emission_prob = normalized_emission_matrix[tag].get(word, 0)
            prob = transition_prob * emission_prob * previous_probabilities[previous_tag]
            print(f"prob of {word} and for {tag} = ", transition_prob ,"*", emission_prob ,"*", previous_probabilities[previous_tag])
            print(f"prob of {word} and for {tag} = ",transition_prob * emission_prob * previous_probabilities[previous_tag])

            if prob > current_probabilities[tag]:
                current_probabilities[tag] = prob
                best_tags[tag] = previous_tag

        best_tag = max(current_probabilities, key=current_probabilities.get)
        tags.append(best_tag)

        previous_probabilities[best_tag] = current_probabilities[best_tag]
        previous_tag = best_tag

    return tags

sentence_to_tag = ['Justin', 'will', 'spot', 'Will']
tagged_output = tag_sentence(sentence_to_tag)

print("Sentence:", " ".join(sentence_to_tag))
print("Tags:", " ".join(tagged_output))



Emission Matrix:
               Justin         Will           Spot           pat            can            watch          Martin         spot           will           
N              0.2222         0.0000         0.2222         0.0000         0.0000         0.0000         0.4444         0.0000         0.1111         
M              0.0000         0.2500         0.0000         0.0000         0.2500         0.0000         0.0000         0.0000         0.5000         
V              0.0000         0.0000         0.0000         0.2500         0.0000         0.5000         0.0000         0.2500         0.0000         
<s>            0.0000         0.0000         0.0000         0.0000         0.0000         0.0000         0.0000         0.0000         0.0000         
</s>           0.0000         0.0000         0.0000         0.0000         0.0000         0.0000         0.0000         0.0000         0.0000         

Transition Matrix:
               N              M              V          